In [37]:
import pandas as pd
import requests
from bs4 import BeautifulSoup


def get_character_info(url):
    response = requests.get(url)

    soup = BeautifulSoup(response.text, 'html.parser')

    # nome = soup.select('h2.pi-item.pi-item-spacing.pi-title.pi-secondary-background')[0].text
    columns = soup.select('h3.pi-data-label.pi-secondary-font')
    infos = []
    for el in soup.select('div.pi-data-value.pi-font'):

        infos.append(el.text)

    result = pd.DataFrame([infos], columns=[el.text for el in columns])
    # result = result.assign(Nome = nome)
    return result


In [45]:
get_character_info('https://harrypotter.fandom.com/pt-br/wiki/Argo_Filch')


,Nascimento,Estado sanguíneo,Espécie,Gênero,Cor do cabelo,Cor da pele,Varinha,Lealdade
0,1956,Aborto,Humano,Masculino,Cinza,Pálida,Não Possui,Escola de Magia e Bruxaria de HogwartsBrigada ...


# function to take a txt file with all links and concat everyone

In [53]:
def get_info_from_file(file_name):
    with open(file_name, 'r') as f:
        urls = f.readlines()

    full_df = pd.DataFrame()
    for url in urls:
        url = url.replace('\n', '')
        full_df = pd.concat([full_df, get_character_info(url)])

    return full_df


get_info_from_file('links_personagens.txt')


,Nascimento,Estado sanguíneo,Espécie,Gênero,Cor do cabelo,Cor da pele,Varinha,Lealdade,Estado civil,Nacionalidade,...,Assinatura,Altura,Peso,Cor dos olhos,Membros da família,Romances,Bicho-papão,Patrono,Ocupação,Casa
0,1956,Aborto,Humano,Masculino,Cinza,Pálida,Não Possui,Escola de Magia e Bruxaria de HogwartsBrigada ...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,"19 de setembro de 1979[1], Inglaterra[2], Grã-...",Nascida trouxa[3],Humana[17],Feminino[17],Castanho[18],Clara[21][22],"Videira, fibra de coração de dragão e vinte e ...",Família GrangerFamília WeasleyFamília PotterEs...,Casada[4][5],Inglesa,...,,"1,65cm[2]",53kg[2],Castanho[19][20],Sr. Granger (pai)Sra. Granger (mãe)Hector Dagw...,Vítor Krum (anteriormente)Córmaco McLaggen (an...,Falhar [23],Lontra[24][25],Funcionária do Departamento para Regulamentaçã...,Grifinória[27]


In [54]:
jorge_weasley = 'https://harrypotter.fandom.com/pt-br/wiki/Jorge_Weasley'


get_character_info(jorge_weasley)


,Nascimento,Estado sanguíneo,Estado civil,Nacionalidade,Outros nomes,Título(s),Assinatura,Espécie,Gênero,Altura,Cor do cabelo,Cor da pele,Membros da família,Romances,Bicho-papão,Varinha,Patrono,Ocupação,Casa,Lealdade
0,"1 de abril de 1978, Grã-Bretanha[1]",Sangue puro[2],Casado,Inglês,Jred[2]Jorgie (por Molly Weasley)Harry Potter ...,Batedor[2],,Humana,Masculino,"Abaixo de 1,52 [5]",Ruivo[6],Branca,Septimus Weasley (avô paterno) †Cedrella Weasl...,Angelina Johnson (esposa),Lorde Voldemort[8],"Comprimento, madeira e núcleo desconhecidos",Magpie[9],Co-proprietário das Gemialidades Weasley [10],Grifinória,Família WeasleyFamília PrewettEscola de Magia ...


# get links of all characters

In [109]:
link_test = 'https://harrypotter.fandom.com/pt-br/wiki/Categoria:Personagens'
response = requests.get(link_test)
soup = BeautifulSoup(response.text, 'html.parser')

# get all links for characters
links_personagens = []
for c in soup.select('ul.category-page__members-for-char a.category-page__member-link'):
    if 'Categoria' in c['href']:
        continue
    links_personagens.append(c['href'])
    # break

url_personagem_base = 'https://harrypotter.fandom.com/'

links_personagens_completos = []
for link in links_personagens:
    links_personagens_completos.append(url_personagem_base + link)

# save to txt

with open('links_personagens.txt', 'a') as f:
    for link in links_personagens_completos:
        f.write(link + '\n')

In [110]:
get_info_from_file('links_personagens.txt')

,Born,Died,Blood status,Nationality,Species,Gender,Hair colour,Skin colour,Family members,Boggart,...,Eye colour,Wand,Marital status,Also known as,Title(s),Height,Weight,Romances,Patronus,Occupation
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,Between 3 May and 31 August 1981[1],"2 May 1998 (aged 16)Hogwarts Castle, Highlands...",Muggle-born[2],British or Irish[3],Human,Male,Mousy[4],Light (likely)[5],FatherDennis Creevey (younger brother)Creevey ...,Lord Voldemort[6],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,between 1 September 1982 and 31 August 1983Gre...,NaN,Pure-blood or half-blood[2],British or Irish[3],Human[4],Male[4],Strawberry blond[4],Light[4],Jamie Wolpert (possible relative),NaN,...,Blue[4],"Unknown length, wood and core",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,"1 March 1980Ottery St Catchpole, Devon, Englan...",NaN,Pure-blood[2],English,Human[11],Male[11],Red[12][13],Pale[15][16][17][18][19][20][21],Arthur Weasley (father)Molly Weasley (née Prew...,Aragog[23][6],...,Blue[14],"12"", Ash, unicorn tail hair (formerly, broken)...",Married[3],Ron (by family & friends)Ickle Ronnikins (by F...,Prefect[7]Keeper[7],"5' 9""[1]",152 lbs[1],Lavender Brown (ex-girlfriend) †Hermione Grang...,Jack Russell terrier[24],Auror (formerly)Weasleys' Wizard Wheezes co-ma...
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,1 April 1978Great Britain,"2 May 1998 (aged 20)Seventh-floor corridor, Ho...",Pure-blood,English,Human[5],Male[5],Red[6],White,Arthur Weasley (father)Molly Weasley (née Prew...,Lord Voldemort[7],...,NaN,"Unknown length, wood and core",Single,Rodent (Potterwatch)[2]Rapier (Potterwatch)[2]...,Beater[5],NaN,NaN,Angelina Weasley née Johnson (date),Magpie[8],Co-owner of Weasleys' Wizard Wheezes[9]
